In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import necessary modules from LangChain
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import os

In [3]:
FILE_WITH_SECRET = '/Users/neel/Documents/Keys/OpenAIDeployWizardKey.txt'
MODEL_NAME = 'gpt-4o-mini'

# Overview Chat UI
1. User will tell this is the model and this is the inference script which needs to be deployed on EC2 as an api server
2. We productionize the inference script (adding logging + exception hanfling)
3. We generate docker file for this (there is a conversation fo pulling the model from a model registry, but that is out of scope for the poc)
4. Terraform code for spinnning the infra
5. a shell script or commands that can be run on the instance 

In [4]:
# Function to read OpenAI API key from a file
def read_openai_key(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.read().strip()
    except Exception as e:
        return str(e)

# Set your OpenAI API key from a file
api_key = read_openai_key(FILE_WITH_SECRET)
os.environ["OPENAI_API_KEY"] = api_key

# Create a ChatOpenAI instance
chat = ChatOpenAI(model=MODEL_NAME, temperature=0)


# Initialize an empty list to store the chat history
messages = []
# File writer instance
# file_writer = FileWriterTool()

# # Start an infinite loop for continuous conversation
# while True:
#     # Get user input
#     message = input("> ")
    
#     # Check if the user wants to exit the chat
#     if message.lower() == "exit":
#         print("Exiting chat. Goodbye!")
#         break
    
#     # Create a HumanMessage object with the user input
#     usr_msg = HumanMessage(content=message)
    
#     # Append the user message to the messages list
#     messages.append(usr_msg)
    
#     # Generate a response from the AI using the chat history
#     ai_msg = chat(messages)
    
#     # Print the AI's response
#     print(ai_msg.content)
    
#     # Append the AI's response to the messages list
#     messages.append(ai_msg)

/var/folders/lr/g8_szk2d7690c34fwxhrsgb00000gn/T/ipykernel_10662/1532119186.py:14: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(model=MODEL_NAME, temperature=0)


## Containerizing model and inference script

## Terraform Code generation for EC2 instance

In [ ]:
# Define parameters in a dictionary
params = {
    "vpc_id": "vpc-0f0aea174086b6625",
    "region": "us-west-1",
    "subnet_id": "subnet-070d54662e68443ed",
    "instance_type": "t2.micro",
    "security_group_id": "sg-03aa3023dd84cf4a5",
    "key_pair_name": "neel_test",
    "ami_id": "ami-08d4f6bbae664bd41"
}

# Create the terraform prompt using dictionary formatting
terraform_prompt = '''Role: Terraform DevOps Engineer  
Task: Generate AWS EC2 instance Terraform code using provided parameters.  
Context:  
- Use AWS Provider (version >= 4.0).  
- Preconfigured resources: VPC, security group, and AMI (see details below).  
- Missing values (subnet, key_pair) must be flagged with a `TODO` comment.  
- Ensure security best practices (e.g., no hardcoded secrets).  

Action:  
1. Define a Terraform `aws_instance` resource with the provided configuration.  
2. Reference existing resources (VPC, security group, Key value pair etc).  
3. Add tags for resource tracking.  
4. Include error handling for missing subnet/key pair.  
5. If value of a parameter is known, replace the placeholder with the actual value.

Output Format:  
- Provide valid HCL (HashiCorp Language) code.  
- Enclose the complete Terraform code in triple backticks (```), prefixed with "```hcl" for syntax highlighting, to allow easy extraction by Python scripts.  
- When providing code for multiple files such as main.tf, variables.tf, etc., separate them with a comment line (e.g., `# variables.tf`).
- Include comments for placeholders (e.g., `# TODO: Replace with your subnet ID`).  
- Use a separate `variables.tf` for configurable values (e.g., `var.instance_type`).  

Parameters Provided:  
vpc_id = "{vpc_id}"  
subnet_id = "{subnet_id}"  
instance_type = "{instance_type}"  
security_group_id = "{security_group_id}"  
key_pair_name = "{key_pair_name}"  
ami_id = "{ami_id}"  
region = "{region}"
'''.format(**params)

print(terraform_prompt)


Role: Terraform DevOps Engineer  
Task: Generate AWS EC2 instance Terraform code using provided parameters.  
Context:  
- Use AWS Provider (version >= 4.0).  
- Preconfigured resources: VPC, security group, and AMI (see details below).  
- Missing values (subnet, key_pair) must be flagged with a `TODO` comment.  
- Ensure security best practices (e.g., no hardcoded secrets).  

Action:  
1. Define a Terraform `aws_instance` resource with the provided configuration.  
2. Reference existing resources (VPC, security group, Key value pair etc).  
3. Add tags for resource tracking.  
4. Include error handling for missing subnet/key pair.  
5. If value of a parameter is known, replace the placeholder with the actual value.

Output Format:  
- Provide valid HCL (HashiCorp Language) code.  
- Enclose the complete Terraform code in triple backticks (```), prefixed with "```hcl" for syntax highlighting, to allow easy extraction by Python scripts.  
- When providing code for multiple files such 

In [46]:
messages = [
    HumanMessage(content="Hello, I need help with Terraform code generation."),
    HumanMessage(content=terraform_prompt)
]

In [42]:
response = chat.invoke(terraform_prompt)

In [43]:
response_str = response.content
print(response_str)

```hcl
# main.tf
provider "aws" {
  region = "us-east-1"  # TODO: Replace with your desired AWS region
  version = "~> 4.0"
}

resource "aws_instance" "my_instance" {
  ami           = "ami-08d4f6bbae664bd41"
  instance_type = var.instance_type
  subnet_id     = "subnet-070d54662e68443ed"  # TODO: Replace with your subnet ID
  key_name      = "neel_test"  # TODO: Ensure this key pair exists in your account
  vpc_security_group_ids = ["sg-03aa3023dd84cf4a5"]

  tags = {
    Name        = "MyEC2Instance"
    Environment = "Development"
  }

  lifecycle {
    create_before_destroy = true
  }
}

# variables.tf
variable "instance_type" {
  description = "The type of instance to create"
  type        = string
  default     = "t2.micro"
}

# outputs.tf
output "instance_id" {
  description = "The ID of the EC2 instance"
  value       = aws_instance.my_instance.id
}

output "public_ip" {
  description = "The public IP of the EC2 instance"
  value       = aws_instance.my_instance.public_ip
}
```

In [36]:
response_str


'Here is the Terraform code to create an AWS EC2 instance using the provided parameters. The code includes comments for any missing values and follows best practices for security and resource management.\n\n### `main.tf`\n```hcl\nprovider "aws" {\n  region = "us-east-1"  # TODO: Replace with your desired AWS region\n}\n\nresource "aws_instance" "my_instance" {\n  ami           = "ami-08d4f6bbae664bd41"  # Provided AMI ID\n  instance_type = var.instance_type         # Referencing variable for instance type\n  subnet_id     = "subnet-070d54662e68443ed"  # Provided Subnet ID\n  key_name      = "neel_test"               # Provided Key Pair Name\n  vpc_security_group_ids = ["sg-03aa3023dd84cf4a5"]  # Provided Security Group ID\n\n  tags = {\n    Name        = "MyEC2Instance"\n    Environment = "Development"  # TODO: Adjust as necessary\n  }\n\n  # Error handling for missing subnet and key pair\n  lifecycle {\n    prevent_destroy = true\n  }\n}\n\noutput "instance_id" {\n  value = aws_instan

In [49]:
from langchain.schema import AIMessage

In [50]:
messages.append(AIMessage(content=response_str))

In [44]:
import os
import re

def extract_and_save_hcl_files(response_str, folder="aws_terraform_files"):
    """
    Extracts HCL code blocks from the given string and saves each block
    to one or more files in the specified folder. A single HCL code block can
    contain multiple file definitions if it includes multiple header lines,
    each starting with '#' (e.g., "# main.tf").

    :param response_str: The string containing HCL code blocks.
    :param folder: The folder where the files will be saved (default is "aws_terraform_files").
    """
    # Ensure the target folder exists.
    os.makedirs(folder, exist_ok=True)
    
    # Regular expression to capture all HCL code blocks (including newline characters)
    pattern = r"```hcl\s*(.*?)```"
    blocks = re.findall(pattern, response_str, re.DOTALL)

    for block in blocks:
        lines = block.strip().splitlines()
        current_filename = None
        current_lines = []
        
        for line in lines:
            stripped_line = line.strip()
            # Check if the line is a header line indicating a file name
            if stripped_line.startswith("#"):
                # If we were already collecting content for a file, write it out first
                if current_filename is not None:
                    file_path = os.path.join(folder, current_filename)
                    with open(file_path, "w") as f:
                        f.write("\n".join(current_lines).rstrip())
                    print(f"Wrote file: {file_path}")
                # Start a new file: extract the file name (remove '#' and extra whitespace)
                current_filename = stripped_line[1:].strip()
                current_lines = []
            else:
                # If no header has been encountered yet, default to "output.txt"
                if current_filename is None:
                    current_filename = "output.txt"
                current_lines.append(line)
        
        # After processing all lines in the block, write out the last file content.
        if current_filename is not None:
            file_path = os.path.join(folder, current_filename)
            with open(file_path, "w") as f:
                f.write("\n".join(current_lines).rstrip())
            print(f"Wrote file: {file_path}")

In [45]:
extract_and_save_hcl_files(response_str, folder="aws_terraform_files")

Wrote file: aws_terraform_files/main.tf
Wrote file: aws_terraform_files/variables.tf
Wrote file: aws_terraform_files/outputs.tf


In [56]:
terraform_deployment_prompt = f'''
Role: Terraform DevOps Engineer
Task: Generate a series of deployment commands for Terraform code deployment.
Context:
	•	The Terraform configuration files (e.g., main.tf, variables.tf, etc.) have been generated in a previous conversation (see provided context).
	•	The configuration includes AWS provider settings, an AWS EC2 instance resource definition, variable definitions, and error handling for missing parameters.
	•	Deployment should follow Terraform best practices (e.g., initializing the backend, planning, applying, and validating the configuration).

Action:
	1.	List all necessary Terraform commands in the proper order (for example: terraform init, terraform validate, terraform plan, and terraform apply).
	2.	Include any additional pre-deployment or post-deployment commands if applicable (e.g., setting environment variables or using terraform destroy when needed).
	3.	Provide inline comments or brief explanations for each command to guide the user.
	4.	Ensure the commands are tailored to deploy the previously generated Terraform configuration.
    5. Provide steps for validating the deployment after applying the Terraform configuration.
    6. Provie command for connecting to the EC2 instance using SSH.

Output Format:
	•	Enclose the complete command list in triple backticks with “```bash” for shell syntax highlighting.
	•	Each command should appear on its own line with clear inline comments if needed.

Chain Conversation:
	•	Use the previous conversation context—which includes the Terraform code generation prompt and the resulting files—to tailor the deployment commands accordingly.
“””'''

In [57]:
messages.append(HumanMessage(content=terraform_deployment_prompt))
response2 = chat.invoke(messages)

In [58]:
response2_str = response2.content

In [59]:
print(response2_str)

```bash
# Step 1: Initialize the Terraform working directory
terraform init  # Initializes the backend and downloads the necessary provider plugins

# Step 2: Validate the Terraform configuration files
terraform validate  # Checks the configuration files for syntax errors and other issues

# Step 3: Plan the deployment
terraform plan  # Creates an execution plan, showing what actions will be taken without applying them

# Step 4: Apply the Terraform configuration
terraform apply  # Applies the changes required to reach the desired state of the configuration

# Step 5: Validate the deployment
# After applying, you can check the status of the EC2 instance using the AWS CLI or AWS Management Console
aws ec2 describe-instances --instance-ids $(terraform output -raw instance_id)  # Retrieves details of the created EC2 instance

# Step 6: Connect to the EC2 instance using SSH
ssh -i /path/to/your/private-key.pem ec2-user@$(terraform output -raw public_ip)  # Replace with the path to your pri

In [ ]:
# and we can ask provide scripts for running the ec2 instance
